# Audio Generative Aversarial Network

In this lab, we are going to create our first GAN for audio generation.
We are going to use the following dataset:

https://zenodo.org/records/2552860#.XFD05fwo-V4

First, let's import the required libraries:

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchaudio
import IPython.display as ipd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import os


Use Drive to load our data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Select your target data folder:

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks')

To download our data, we can use *wget* (see https://www.gnu.org/software/wget/).

In [ ]:
!wget https://zenodo.org/records/2552860/files/FSDKaggle2018.audio_train.zip?download=1

We also need metadata because the training folder contains several musical instrument audios.
With this, we can decide what we want to generate (a Trumpet, a Tamburine, etc.)

In [ ]:
!wget https://zenodo.org/records/2552860/files/FSDKaggle2018.meta.zip?download=1

Downloaded data are *zip*, let's uncompress them:

In [ ]:
import os
import torch
import zipfile

In [ ]:
# Define file paths
dataset_zip_path_meta = "/content/drive/MyDrive/Colab Notebooks/FSDKaggle2018.meta.zip?download=1"
dataset_zip_path_audio = "/content/drive/MyDrive/Colab Notebooks/FSDKaggle2018.audio_train.zip?download=1"
extract_path_meta = "/content/FSDKaggle2018.meta"
extract_path_audio = "/content/FSDKaggle2018.audio_train"

# Unzip meta file
with zipfile.ZipFile(dataset_zip_path_meta, 'r') as zip_ref:
    zip_ref.extractall(extract_path_meta)

# Unzip audio file
with zipfile.ZipFile(dataset_zip_path_audio, 'r') as zip_ref:
    zip_ref.extractall(extract_path_audio)

Now, let's set our device.
For training a gain we need GPU or MPS.

In [ ]:
def get_device():
    if torch.has_mps:
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

device = get_device()
print(f"Using device: {device}")


In the following, we use *Pandas* to load the metadata (*train.csv*) and filter the instrument we want to focus on.

In [ ]:
import numpy as np
import pandas as pd
import torchaudio
import os
import torch

# Audio Config
DURATION = 4
SAMPLE_RATE = 16000
AUDIO_SHAPE = SAMPLE_RATE * DURATION
MFCC = 40

# Paths
AUDIO_PATH = "/content/FSDKaggle2018.audio_train/FSDKaggle2018.audio_train"
META_PATH = "/content/FSDKaggle2018.meta/FSDKaggle2018.meta/train_post_competition.csv"

# Load Data
def load_train_data(label, input_length=AUDIO_SHAPE):
    # Read metadata
    train = pd.read_csv(META_PATH)

    # Check if label exists
    if label not in train['label'].unique():
        print(f"Label '{label}' not found in the dataset.")
        return np.empty((0, input_length))

    # Filter by label
    train_list = train.loc[train.label == label]
    train_fname_list = train_list.fname

    # Check existing audio files
    existing_audio = []
    for train_fname in train_fname_list:
        file_path = os.path.join(AUDIO_PATH, train_fname)
        if os.path.isfile(file_path):
            existing_audio.append(train_fname)
        else:
            print(f"File not found: {file_path}")

    if not existing_audio:
        print("No valid audio files found.")
        return np.empty((0, input_length))

    X = np.empty((len(existing_audio), input_length))

    # Load and preprocess audio files
    for i, train_fname in enumerate(existing_audio):
        file_path = os.path.join(AUDIO_PATH, train_fname)

        # Read and resample the audio
        data, _ = torchaudio.load(file_path)
        data = data.mean(dim=0).numpy()  # Convert to mono

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length + offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, input_length - len(data) - offset), "constant")

        X[i,] = data

    print(f"Data Loaded: {X.shape[0]} samples")
    return X

# Standardize Data
def normalization(X):
    mean = X.mean(keepdims=True)
    std = X.std(keepdims=True)
    X = (X - mean) / std
    print("Data Normalized...")
    return X

# Rescale Data to be in range [rangeMin, rangeMax]
def rescale(X, rangeMin=-1, rangeMax=1):
    maxi = X.max()
    mini = X.min()
    X = np.interp(X, (mini, maxi), (rangeMin, rangeMax))
    print("Data Rescaled...")
    return X

train_data = normalization(load_train_data("Tambourine", AUDIO_SHAPE))
x_train = rescale(train_data)
print('Audio shape: ', AUDIO_SHAPE)


In [ ]:
x_train.shape

## Define the GAN

We define the Generator and Discriminator.

*Remember:* Generator has to produce a waveform starting from a noise in input. Discriminator is simply a binary classifier (input sample is *real* or *fake*)

In [ ]:
class Generator(nn.Module):
    def __init__(self, nz, ngf):
        super(Generator, self).__init__()

        assert(ngf % 32 == 0 and ngf >= 32)

        self.main = nn.Sequential(
            nn.ConvTranspose1d(
                in_channels=nz,
                out_channels=ngf,
                kernel_size=64,
                stride=1,
                padding=0,
                dilation=1,
                bias=False,
            ),
            nn.BatchNorm1d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(p=0.2),

            nn.ConvTranspose1d(
                in_channels=ngf,
                out_channels=1,
                kernel_size=64,
                stride=2,
                padding=0,
                dilation=1,
                bias=False,
            ),

            nn.Linear(388, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, AUDIO_SHAPE)
        )

    def forward(self, input):
        return self.main(input)




class Discriminator(nn.Module):
    def __init__(self, input_dim=AUDIO_SHAPE):
        super(Discriminator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)


# Initialize models
generator = Generator(1, 512).to(device)
#generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and Optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=3e-4, betas=(0.5, 0.999))


Our GAN is a NN including both the Generator and Discriminator.
We train it and, then, we use only the Generator.

In [ ]:
class GAN(nn.Module):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, x):
        generated_data = self.generator(x)
        return self.discriminator(generated_data)

gan = GAN(generator, discriminator).to(device)


For our Generator we need to produce noise as input.
Let's create some support function: one to generate noise, the other to see and hear generated samples.
Then, let's execute our training

In [ ]:
def sample_noise(size, noise_latent_dim):
    z = torch.FloatTensor(size, noise_latent_dim).to(device)
    z.data.normal_()  # generating latent space based on normal distribution
    #z = torch.zeros(size, noise_latent_dim).to(device)
    return z

def show_gen_samples(generator, epoch, sample_rate=SAMPLE_RATE, samples=3):
    sample_plot = []
    fig = plt.figure(figsize=(1, samples))
    noise = torch.randn(samples, 100, device=device)
    noise = noise.unsqueeze(1)
    #noise = sample_noise(samples, 100)
    audios = generator(noise).detach().cpu().numpy()
    #audios = audios.squeeze(1)
    for i, audio in enumerate(audios):
        ipd.display(ipd.Audio(data=audio, rate=sample_rate))
        sample_plot.append(fig.add_subplot(1, samples, i+1))
        sample_plot[i].plot(audio.flatten(), '-')
    plt.gcf().set_size_inches(25, 5)
    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    fig.suptitle(f"{epoch} Epochs Result", fontsize=17)
    plt.show()

# Training the GAN
D_loss = []
G_loss = []
epochs = 10000
batch_size = 128

for epoch in range(epochs):
    for _ in range(1):
        # Train Discriminator
        real_data = torch.tensor(x_train[np.random.randint(0, x_train.shape[0], batch_size)], dtype=torch.float32).to(device)
        noise = torch.randn(batch_size, 100, device=device)
        #noise = sample_noise(batch_size, 100)
        #real_data = real_data.unsqueeze(1)
        noise = noise.unsqueeze(1)
        #print(noise.shape)

        fake_data = generator(noise)
        fake_data = fake_data.squeeze(1)
        #print(fake_data.shape)
        optimizer_D.zero_grad()

        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        outputs_real = discriminator(real_data)
        outputs_fake = discriminator(fake_data.detach())

        d_loss_real = criterion(outputs_real, real_labels)
        d_loss_fake = criterion(outputs_fake, fake_labels)
        d_loss = d_loss_real + d_loss_fake / 2
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()

        outputs = discriminator(fake_data)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

        D_loss.append(d_loss.item())
        G_loss.append(g_loss.item())

    if (epoch % 100 == 0) or (epoch == epochs-1):
        print(f"Epoch [{epoch+1}/{epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}")

    if epoch % 100 == 0:
        show_gen_samples(generator, epoch)


In [ ]:
show_gen_samples(generator, epochs)

# Generator And Discriminator Loss During Training



In [ ]:
# Training the GAN with adjustments
D_loss = []
G_loss = []
epochs = 500
batch_size = 128

for epoch in range(epochs):
    for _ in range(1):
        # Train Discriminator
        real_data = torch.tensor(x_train[np.random.randint(0, x_train.shape[0], batch_size)], dtype=torch.float32).to(device)
        noise = torch.randn(batch_size, 100, device=device)
        noise = noise.unsqueeze(1)

        fake_data = generator(noise)
        fake_data = fake_data.squeeze(1)
        optimizer_D.zero_grad()

        real_labels = torch.ones(batch_size, 1, device=device) * 0.9  # Label smoothing
        fake_labels = torch.zeros(batch_size, 1, device=device) + 0.1  # Label smoothing

        outputs_real = discriminator(real_data)
        outputs_fake = discriminator(fake_data.detach())

        d_loss_real = criterion(outputs_real, real_labels)
        d_loss_fake = criterion(outputs_fake, fake_labels)
        d_loss = (d_loss_real + d_loss_fake) / 2
        d_loss.backward()
        optimizer_D.step()

        # Train Generator multiple times for every step of Discriminator
        for _ in range(3):
            optimizer_G.zero_grad()
            noise = torch.randn(batch_size, 100, device=device)
            noise = noise.unsqueeze(1)
            fake_data = generator(noise)
            fake_data = fake_data.squeeze(1)

            outputs = discriminator(fake_data)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            optimizer_G.step()

        D_loss.append(d_loss.item())
        G_loss.append(g_loss.item())

    if (epoch % 100 == 0) or (epoch == epochs-1):
        print(f"Epoch [{epoch+1}/{epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}")

    if epoch % 100 == 0:
        show_gen_samples(generator, epoch)

# Plot the losses
plot_losses(D_loss, G_loss)


# Generator And Discriminator Loss During Training

Here the discriminator is more strong than before

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define a stronger discriminator
class StrongDiscriminator(nn.Module):
    def __init__(self, input_dim=AUDIO_SHAPE):
        super(StrongDiscriminator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.4),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

# Initialize models
generator = Generator(1, 256).to(device)  # Reduced generator capacity
discriminator = StrongDiscriminator().to(device)  # Increased discriminator capacity

# Loss and Optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.999))

# Training the GAN with a stronger discriminator
D_loss = []
G_loss = []
epochs = 500
batch_size = 128

for epoch in range(epochs):
    for _ in range(3):  # Train discriminator more frequently
        # Train Discriminator
        real_data = torch.tensor(x_train[np.random.randint(0, x_train.shape[0], batch_size)], dtype=torch.float32).to(device)
        noise = torch.randn(batch_size, 100, device=device)
        noise = noise.unsqueeze(1)

        fake_data = generator(noise)
        fake_data = fake_data.squeeze(1)
        optimizer_D.zero_grad()

        real_labels = torch.ones(batch_size, 1, device=device) * 0.9  # Label smoothing
        fake_labels = torch.zeros(batch_size, 1, device=device) + 0.1  # Label smoothing

        outputs_real = discriminator(real_data)
        outputs_fake = discriminator(fake_data.detach())

        d_loss_real = criterion(outputs_real, real_labels)
        d_loss_fake = criterion(outputs_fake, fake_labels)
        d_loss = (d_loss_real + d_loss_fake) / 2
        d_loss.backward()
        optimizer_D.step()

    # Train Generator once
    optimizer_G.zero_grad()
    noise = torch.randn(batch_size, 100, device=device)
    noise = noise.unsqueeze(1)
    fake_data = generator(noise)
    fake_data = fake_data.squeeze(1)

    outputs = discriminator(fake_data)
    g_loss = criterion(outputs, real_labels)
    g_loss.backward()
    optimizer_G.step()

    D_loss.append(d_loss.item())
    G_loss.append(g_loss.item())

    if (epoch % 100 == 0) or (epoch == epochs-1):
        print(f"Epoch [{epoch+1}/{epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}")

    if epoch % 1000 == 0:
        show_gen_samples(generator, epoch)

# Plot the losses
plot_losses(D_loss, G_loss)
